In [1]:
import pandas as pd

from tqdm import tqdm

In [2]:
def calcs(y):
    games = pd.read_csv('processed/pbp-summary-{}.csv'.format(y), dtype={'GAME_ID':str})
    winners = pd.read_csv('processed/winners-{}.csv'.format(y), dtype={'GAME_ID':str})
    together = pd.merge(games, winners).sort_values(['GAME_ID','EVENTNUM'])
    compiled = []
    for i in range(1, together.POINTS.max() + 1):
        subset = together[together.POINTS >= i].drop_duplicates(subset=['GAME_ID',], keep='first')
        games = len(subset)
        correct = (subset.SIDE == subset.WINNER).sum()
        pct_correct = correct / float(games)
        compiled.append([i, games, correct, pct_correct])
    return compiled

In [3]:
byyears = {y: calcs(y) for y in tqdm(range(1997,2020))}

100%|██████████| 23/23 [00:52<00:00,  2.10s/it]


In [4]:
alldata = []
for i in byyears:
    data = byyears[i]
    for row in data:
        entry = [i]
        entry.extend(row)
        alldata.append(entry)

In [5]:
final = pd.DataFrame(alldata, columns=['year','point','games','correct','pct_correct'])

In [6]:
final.to_csv('point-tables.csv', index=False)